In [ ]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Überprüfen der $S_n$ Mengen

Bei der Bestimmung der $S_n$ Mengen wurde die Retrograde Analyse verwendet.
Um sicherzustellen, dass die Bretter die bspw. in $S_{10}$ zu finden sind, auch wirklich in <=10 Zügen beendet werden, wurden in diesem Notebook Funktionen definiert, die diesen Aspekt überprüfen. Hierfür werden zuerst die berechneten $S_n$ Mengen aus der Datei geladen.

In [ ]:
S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_fens("S_n_seq_rook")

Die erste Funktion, die zur Überprüfung definiert wurde, lautet ``fen_in_lower_sequence``. Diese überprüft für eine mitgegebene ``fen``, ob diese in einer $S_n$ Menge liegt, für die gilt: 

$n < sequence\_index$. 

Die Funktion gibt hierfür einen booleschen Wert zurück.

In [ ]:
def fen_in_lower_sequence(fen, sequence_index, s_n_short):
    for s in s_n_short[:sequence_index]:
        if fen in s:
            return True
    return False

Als ersten Schritt wird überprüft, dass keine der Schachbretter in eine zu niedrige $S_n$ Menge eingeordnet wurde. Dies wird mit der Funktion ``every_move_of_sequence_in_lower`` umgesetzt. Hierfür erhält sie eine ``sequence_index``, die überprüft werden soll. Für jede Spielsituation wird folgendes überprüft:

$$\forall board \in S_n : move \in board.legal\_moves \implies board.push(move) \in S_m, m<n$$

Als Ergebnis gibt die Funktion eine boolesche Variable zurück, die ``True`` zurückgibt, falls die Bedingung erfüllt ist, ansonsten ``False``.

In [ ]:
def every_move_of_sequence_in_lower(sequence_index, s_n_fen, s_n_short):
    print("Checking S_" + str(sequence_index) + "...")
    sequence_fen = s_n_fen[sequence_index]
    for fen in sequence_fen:
        cur_board = chess.Board(fen)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if not fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    print("Fen: " + cur_short + " not in lower S")
                    return False
                cur_board.pop()
            return True
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    in_lower = True
                cur_board.pop()
            return in_lower


In [ ]:
for i in range(len(S_N_Sequence_fen)):
    every_move_of_sequence_in_lower(i, S_N_Sequence_fen, S_N_Sequence_fen_short)

Die Funktion ``find_move_count`` kann für folgende Überprüfung verwendet werden:

$ \forall board \in S_n \implies max(find\_move\_count(board)) = n $

Hierbei berechnet ``find_move_count()`` jeden Ablauf einer Schach-Partie bis ein Schachmatt erzielt worden ist. Da aber die Spielsituation in die Menge $S_n$ zugeordnet wurde, muss auf jeden Fall eine dieser Zahlen den Wert n betragen, da ansonsten das Schachbrett der falschen $S_n$ Menge zugeordnet wurde.

Die Implementierung der Funktion ``find_move_count`` erfolgt rekursiv. Die Funktion ruft sich nämlich solange selbst auf, bis ein Schachmatt erzielt worden ist. Für einen derartigen Aufruf wird die ``sequence_index``, in der sich die ``fen`` befindet, mitgegeben. Außerdem muss das ``move_set``, das auch als Rückgabewert fungiert, mitgegeben werden, da darin die Anzahl der Züge gespeichert werden, die zum Lösen jeglicher Spielabläufe benötigt werden. Der Parameter ``move_count`` resultiert ebenfalls durch die Rekursion, da in ihr die aktuelle Anzahl an ausgeführten Spielzügen gespeichert wird. Aufgrund des Aufrufs der Funktion ``find_situation_in_sequence`` wird zusätzlich noch die Liste der gekürzten $S_n$ Mengen beigefügt (``s_n_short``). Die Funktion gibt letztendlich eine Menge an Spielzügen zurück, die für die unterschiedlichen Abläufe der Spielsituation benötigt wurden.

In [ ]:
def find_move_count(sequence_index, fen, move_count, move_set, s_n_short):
    cur_board = chess.Board(fen)
    moves = cur_board.legal_moves
    if moves.count() == 0 and sequence_index == 0:
        move_set.add(move_count)
        return move_set
    if not cur_board.turn:
        # print("Black:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            move_count += 1
            cur_fen = cur_board.fen()
            new_seq = find_situation_in_sequence(cur_fen, s_n_short)
            # print(new_seq, ";" , cur_fen)
            move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
            move_count -= 1
            cur_board.pop()
    else: 
        # print("White:")
        for move in cur_board.legal_moves:
            cur_board.push(move)
            cur_fen = get_board_and_turn(cur_board.fen())
            if fen_in_lower_sequence(cur_fen, sequence_index, s_n_short):
                new_seq = find_situation_in_sequence(cur_fen, s_n_short)
                move_count += 1
                move_set = find_move_count(new_seq, cur_fen, move_count, move_set, s_n_short)
                move_count -= 1
            cur_board.pop()
    return move_set

In [ ]:
find_move_count(20, "8/8/8/3K4/8/5R2/2k5/8 b - - 0 1", 0, set(), S_N_Sequence_fen_short)

Zur Überprüfung, dass nun kein Board der falschen $S_n$ Menge zugeordnet wurde, wurde die Funktion ``check_boards_in_correct_sequence`` definiert. Diese überprüft für einen ``sequence_index`` in einer mitgegebenen Liste von $S_n$ Mengen (``s_n_fen``), ob die Spielsituationen der Menge an dem Index ``sequence_index`` auch maximal $n$ Züge bis zum Ende der Partie benötigen. Dies gelingt mit der Funktion ``find_move_count``, die zusätzlich noch die gekürzte Schreibweise der $S_n$ Mengen benötigt (``s_n_short``). Die Funktion gibt in Form einer Konsolenausgabe an, welche der FENs in eine falsche $S_n$ Menge zugeordnet worden sind.

In [ ]:
def check_boards_in_correct_sequence(sequence_index, s_n_fen, s_n_short):
    invalid = 0
    print("Comparing S_" + str(sequence_index) + "...")
    for fen in s_n_fen[sequence_index]:
        move_set = find_move_count(sequence_index, fen, 0, set(), s_n_short)
        if max(move_set) != sequence_index:
            invalid += 1
            print(fen)
    print("Invalid boards: " + str(invalid))

In [ ]:
check_boards_in_correct_sequence(4, S_N_Sequence_fen, S_N_Sequence_fen_short)

Zusätzlich zum rechnerischen überprüfen der Ergebnisse kann eine andere Methode zum verifizieren verwendet werden. 
Schach-Endspieldatenbanken sind keine neue Erfindung und wurden bereits von anderen Forschern entwickelt.
Ein langanhaltendes Projekt, welches gegenwärtig Datenbanken für Situationen mit bis zu 7 Figuren auf dem Spielfeld anbietet ist [Syzygy](https://syzygy-tables.info/).
Das Projekt hält zwei Datentypen für alle Spielsituationen for:
WDL Daten und DTZ Daten.

WDL steht für Win / Draw / Loss und gibt dem Nutzer eine Information über den Wert einer Spielsituation. 
Eine Anfrage an die Datenbank mit einer Situation wird mit einem der folgenden Werte beantwortet: -2, -1, 0, 1, 2.
Positive Werte implizieren, dass bei perfektem Spiel der aktuelle Spieler gewinnnt, negative Werte bedeuten, dass der aktuelle Spieler verliert.
Die 0 bedeutet, dass das Spiel (wenn beide Seiten perfekt spielen) in einem Unentschieden endet.
Eine Zwei ist ein sicherer Sieg / Verlust, während eine Eins in einem Gewinn oder unentschieden mittels der 50-Zug Regel enden kann.

Die interessantere der Dateien ist die DTZ-Datei.
DTZ steht für Distance to Zero. Die DTZ Tabelle enthält Werte von -100 bis 100. 
Positiv, Negativ und Null kann genau wie WDL interpretiert werden.
Die Zahlen von -100 bis -1 und 1 bis 100 geben die Anzahl der Halbzüge bis zu einem Gewinn (oder Reset der 50-Züge Regel) an.
Stetiges verringern einer positiven DTZ führt also zu einem Gewinn.

Die DTZ-Zahl einer Spielsituation kann mit dem $n$ verglichen werden, in welcher Menge $S_n$, diese Situation in der `.chessAI` eingeordnet wurde.
Stimmen diese Zahlen überein, war die Berechnung korrekt.

*Hinweis: Je nach Ausführung der Syzygy-Tabellen werden halbe oder ganze Züge gespeichert, es muss daher beim Vergleich eine Toleranz von einem $n$ akzeptiert werden.*    

Die Funktion `compare_with_syzygy()` führt diesen Vergleich durch und gibt die Anzahl falsch eingeordneter Situationen zurück. 

In [ ]:
def compare_with_syzygy(syzygy, s_n_sequence):
    count = 0
    
    for n in range(len(s_n_sequence)):
        for fen in s_n_sequence[n]:
            chess_board = chess.Board(fen)
            if n != abs(syzygy.probe_dtz(chess_board)) != n + 1:
                count += 1
    print(f"Syzygy believes {count} Situations are wrongly placed in the sequence.")
    return count

Die Syzygy-Dateien müssen sich entweder im Ordner `./syzygy` befinden oder der Pfad angepasst werden.

In [ ]:
SYZYGY = chess.syzygy.Tablebase()
SYZYGY.add_directory("./syzygy")
compare_with_syzygy(SYZYGY, S_N_Sequence_fen)
